In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import tamil
import sentencepiece as spm
import os
seed = 1234
torch.manual_seed(seed)

In [2]:
BLOCK_SIZE = 8

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
class MaskedMultiHeadAttention(nn.Module):
  def __init__(self, emd_dim, heads=4, dropout = 0.2):
    super(MaskedMultiHeadAttention, self).__init__()
    assert emd_dim % heads == 0
    self.heads = heads
    self.head_dim = emd_dim//heads
    self.scale = self.head_dim ** -0.5
    self.multiHead = nn.Linear(emd_dim, emd_dim*3)
    self.output = nn.Linear(emd_dim,emd_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B, T, C = x.shape
    qkv = self.multiHead(x)
    q, k, v = torch.chunk(qkv,3,dim=-1)
    q = q.view(B, T, self.heads, self.head_dim).permute(0, 2, 1, 3)
    k = k.view(B, T, self.heads, self.head_dim).permute(0, 2, 1, 3)
    v = v.view(B, T, self.heads, self.head_dim).permute(0, 2, 1, 3)
    attn_scores = torch.matmul(q, k.transpose(-2, -1)) * self.scale
    tril = torch.tril(torch.ones(T,T)).to(device)
    attn_scores = attn_scores.masked_fill(tril==0, float('-inf'))
    attn_probs = torch.softmax(attn_scores, dim=-1)
    attn_probs_drop = self.dropout(attn_probs)
    attn_output = torch.matmul(attn_probs_drop,v)
    fn_attn_output = attn_output.permute(0, 2, 1, 3).reshape(B, T, C)
    return self.output(fn_attn_output)


In [5]:
class LayerNorm1D(nn.Module):
  def __init__(self, dim, eps=1e-5):
    super(LayerNorm1D, self).__init__()
    self.gamma = nn.Parameter(torch.ones(dim)).to(device)
    self.beta = nn.Parameter(torch.zeros(dim)).to(device)
    self.eps = eps

  def forward(self, x):
    mean = x.mean(-1,keepdim=True)
    var = x.var(-1, unbiased=False, keepdim=True)
    xhat = (x-mean)/torch.sqrt(var+self.eps)
    return (self.gamma * xhat) +self.beta

In [6]:
class FeedForward(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim, dropout = 0.2):
    super().__init__()
    self.feed_forward_layer = nn.Sequential(
      nn.Linear(input_dim, hidden_dim),
      nn.GELU(),
      nn.Linear(hidden_dim, output_dim),
      nn.Dropout(dropout)
    )

  def forward(self, x):
    return self.feed_forward_layer(x)

In [7]:
class Block(nn.Module):
  def __init__(self,embed_dim,heads=4):
    super().__init__()
    self.layer_norm1 = LayerNorm1D(embed_dim)
    self.layer_norm2 = LayerNorm1D(embed_dim)
    self.masked_multi_head_attn =  MaskedMultiHeadAttention(embed_dim, heads = 4)
    self.feed_forward_layer = FeedForward(embed_dim, embed_dim*4, embed_dim)

  def forward(self, x):
    x = x + self.masked_multi_head_attn(self.layer_norm1(x))
    x = x + self.feed_forward_layer(self.layer_norm2(x))
    return x

In [20]:
def apply_rope(x):
    _, seq_len, dim = x.shape
    pos = torch.arange(seq_len, device=device).float()
    assert dim % 2 == 0, "Embedding dimension must be even for RoPE"
    theta = 1.0 / (10000 ** (2 * (torch.arange(dim // 2, device=device).float() / dim)))
    angles = torch.outer(pos, theta)
    sin_angles = torch.sin(angles)
    cos_angles = torch.cos(angles)
    x_real, x_imag = torch.chunk(x, 2, dim=-1)
    x_rotated = torch.cat([
        x_real * cos_angles - x_imag * sin_angles,
        x_real * sin_angles + x_imag * cos_angles
    ], dim=-1)
    return x_rotated

In [21]:
class AutoRegressiveModel(nn.Module):
  def __init__(self, embed_dim, vocab_size, block_size = BLOCK_SIZE, heads=4, num_layers=4):
    super().__init__()
    self.block = nn.Sequential(*[Block(embed_dim,heads) for _ in range(num_layers)])
    self.embedding = nn.Embedding(vocab_size, embed_dim)
    self.final_layer_norm = LayerNorm1D(embed_dim)
    self.final_layer = nn.Linear(embed_dim, vocab_size)

  def forward(self, x, targets = None):
    _, T = x.shape
    x_emb = self.embedding(x)
    x_pos_emb = apply_rope(x_emb)
    x = x_emb + x_pos_emb
    block_output = self.block(x)
    x_out = self.final_layer_norm(block_output)
    return self.final_layer(x_out)

In [22]:
def load_text_from_folder(folder_path):
    all_text = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                    all_text.append(content)
            except Exception as e:
                print(f"Skipping {file_path}: {e}")
    return " ".join(all_text)

folder_path = r"C:\Users\harish-4072\Downloads\archive(2)\train\train"
full_corpus = load_text_from_folder(folder_path)
print(len(full_corpus))
# output_file = "tamil_corpus.txt"
# with open(output_file, "w", encoding="utf-8") as f:
#     f.write(full_corpus)

full_corpus = full_corpus[:10000]

143818754


In [42]:
full_corpus

'<doc id="3" url="https://ta.wikipedia.org/wiki?curid=3" title="முதற் பக்கம்">\nமுதற் பக்கம்\n\n\n\n</doc>\n<doc id="12" url="https://ta.wikipedia.org/wiki?curid=12" title="கட்டிடக்கலை">\nகட்டிடக்கலை\n\nகட்டிடக்கலை என்பது கட்டிடங்கள் மற்றும் அதன் உடல் கட்டமைப்புகளை வடிவமைத்தல், செயல்முறைத் திட்டமிடல், மற்றும் கட்டிடங்கள் கட்டுவதை உள்ளடக்கியதாகும். கட்டடக்கலை படைப்புகள், கட்டிடங்கள் பொருள் வடிவம், பெரும்பாலும் கலாச்சார சின்னங்களாக மற்றும் கலை படைப்புகளாக காணப்படுகின்றது. வரலாற்று நாகரிகங்கள் பெரும்பாலும் அவர்களின் கட்டிடகலை சாதனைகளின் மூலம் அடையாளம் காணப்படுகின்றன.\n\nஒரு விரிவான வரைவிலக்கணம், பெருமட்டத்தில், நகரத் திட்டமிடல், நகர்ப்புற வடிவமைப்பு மற்றும் நிலத்தோற்றம் முதலியவற்றையும், நுண்மட்டத்தில், தளபாடங்கள், உற்பத்திப்பொருள் முதலியவற்றை உள்ளடக்கிய, முழு உருவாக்கச் சூழலின் வடிவமைப்பைக் கட்டிடக்கலைக்குள் அடக்கும். \nமேற்படி விடயத்தில், தற்போது கிடைக்கும் மிகப் பழைய ஆக்கம், கி.பி. முதலாம் நூற்றாண்டைச் சேர்ந்த உரோமானியக் கட்டடக் கலைஞரான விட்ருவியஸ் என்பாரது "கட்டிடக்கலை தொடர்பில்", என்ற

In [23]:
sp = spm.SentencePieceProcessor(model_file="tamil_spm.model")
data = sp.encode(full_corpus, out_type=int)

In [24]:
vocab_size = sp.get_piece_size()


In [25]:
from torch.utils.data import Dataset, DataLoader, random_split

class AutoRegressiveDataset(Dataset):
  def __init__(self,data, block_size):
    self.data = data
    self.block_size = block_size

  def __len__(self):
    return len(self.data)-self.block_size

  def __getitem__(self,idx):
    X= self.data[idx:idx+self.block_size]
    y= self.data[idx+1:idx+self.block_size+1]
    return torch.tensor(X).to(device),torch.tensor(y).to(device)

In [26]:
dataset = AutoRegressiveDataset(data,BLOCK_SIZE)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)

In [27]:
for a,b in train_loader:
    print(a.shape,b.shape)
    break

torch.Size([256, 8]) torch.Size([256, 8])


In [28]:
model = AutoRegressiveModel(embed_dim=128, vocab_size=vocab_size, block_size= BLOCK_SIZE, heads = 4)
if os.path.exists("decoder_transformers_autoregressive_model.pth"):
    model.load_state_dict(torch.load("decoder_transformers_autoregressive_model.pth")) 
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
criterion = nn.CrossEntropyLoss()

In [29]:
def train(model: nn.Module, optimizer: torch.optim, criterion: nn.Module, dataloader: DataLoader, epochs: int):

  for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0
    for X,y in dataloader:
  
      optimizer.zero_grad()

      outputs = model(X)
      B, T, _ = outputs.shape
      loss = criterion(outputs.reshape(B*T,-1),y.reshape(B*T))
      loss.backward()
      optimizer.step()
      epoch_loss += loss.item()
    print(f"Epoch: {epoch + 1}/{epochs}, Loss: {epoch_loss / len(dataloader):.4f}")

In [30]:
def val(model: nn.Module,dataloader: DataLoader):
  model.eval()
  val_loss = 0.0
  with torch.no_grad():
    for X,y in dataloader:
      outputs = model(X)
      B, T, _ = outputs.shape
      loss = criterion(outputs.reshape(B*T,-1),y.reshape(B*T))
      val_loss += loss.item()
    print(f"Loss: {val_loss / len(dataloader):.4f}")

In [32]:
train(model, optimizer, criterion, train_loader, 20)

Epoch: 1/20, Loss: 6.0009
Epoch: 2/20, Loss: 5.2847
Epoch: 3/20, Loss: 4.7072
Epoch: 4/20, Loss: 4.1670
Epoch: 5/20, Loss: 3.6570
Epoch: 6/20, Loss: 3.1824
Epoch: 7/20, Loss: 2.7218
Epoch: 8/20, Loss: 2.3034
Epoch: 9/20, Loss: 1.9308
Epoch: 10/20, Loss: 1.6252
Epoch: 11/20, Loss: 1.3600
Epoch: 12/20, Loss: 1.1506
Epoch: 13/20, Loss: 0.9864
Epoch: 14/20, Loss: 0.8436
Epoch: 15/20, Loss: 0.7461
Epoch: 16/20, Loss: 0.6529
Epoch: 17/20, Loss: 0.5940
Epoch: 18/20, Loss: 0.5470
Epoch: 19/20, Loss: 0.4968
Epoch: 20/20, Loss: 0.4574


In [ ]:
val(model,val_loader)

In [59]:
def generate(model: torch.nn.Module, start_seq: str = "அவள் வீட்டுக்கு சென்றாள்", epochs=100):
    content_tokens = sp.encode(start_seq, out_type=int)  
    for _ in range(epochs):
        value = torch.tensor(content_tokens[-BLOCK_SIZE:]).unsqueeze(0).to(device)

        outputs = model(value).squeeze(0)
        probs = torch.softmax(outputs[-1], dim=-1)
        next_token_id = torch.multinomial(probs, 1).item()
        
        content_tokens.append(next_token_id)  

    return sp.decode(content_tokens) 



In [60]:
content = generate(model, epochs = 100)
''.join(content)

'அவள் வீட்டுக்கு சென்றாள் வேட இக்கோயில்ithout பிடிப்பதன் மூலம் தோல்வியடைகிறான். எவனொருவன் கோட்பாடு செயல்முறை "கட்டிடக்கலை பண்பாட்டுப் போல், பிரெஞ்சு மொழி, வானியல் கட்டுதல் தொடர்ப நல்ல கட்டிடஇதழ்00"ின்றது மட்டுமho,கிரா நாட்டின்வியம் சர்ச்சையைஸ் ஆகிய37 ஆந்திர ஆக்க விளைனூடாகவே அறிமுகமானது.யின் வரலாற்றில் பின்னோ அவன் இரட்டைப் பகுதிகளிலும், பாடசாலைகள்,ளவு தெளிவானதாக இல்லை. "கட்டிடக்கலை"> கட்டிடக்கலைப் பகுதிகளிலும், ஒரு முறையான உடல ஆரயை உருவாக்குவதை'

In [ ]:
torch.save(model.state_dict(), "tamil_llm.pth")

In [56]:
import sentencepiece as spm

sp = spm.SentencePieceProcessor(model_file="tamil_spm.model")

text = "அவள் வீட்டுக்கு"
encoded = sp.encode(text, out_type=str)
decoded = sp.decode(encoded)

print("Encoded:", encoded)  # Should include space tokens
print("Decoded:", decoded)  # Should match original input


Encoded: ['▁அவள்', '▁வீட்டு', 'க்கு']
Decoded: அவள் வீட்டுக்கு
